# 0 环境配置

In [1]:
basedir = "/home/ubuntu/dishui_ddd/input/"

In [2]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

#seed
np.random.seed(7)

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *
from my_image import MyImageDataGenerator
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [3]:
modelname = "M17"

In [4]:
from keras.applications.xception import preprocess_input

In [5]:
# changing
model_image_size = (330, 440)
batch_size = 16

# 2 数据生成

In [6]:
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)

gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = train_gen.flow_from_directory(os.path.join(basedir, 'train_all'),  model_image_size, \
                shuffle=True, batch_size=batch_size, class_mode="categorical", mix_p=1, mix_horizon_p=0.5)

test_generator = gen.flow_from_directory(os.path.join(basedir,"test"),  model_image_size, \
                                         shuffle=False, batch_size=batch_size, class_mode=None)

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


# 3 搭建模型

In [7]:
inputs = Input((*model_image_size, 3))
base_model = Xception(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print("total layer count {}".format(len(base_model.layers)))

total layer count 132


# 4 训练模型

In [8]:
steps_train_sample = int(train_generator.samples*0.35)//batch_size + 1
print("train_generator.samples = {}".format(steps_train_sample))

train_generator.samples = 491


## （1） epoch 1，学习率1e-4，使用Adam

In [9]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1)

Epoch 1/1
491/491 [==============================] - 992s - loss: 0.5394 - acc: 0.8383   


In [10]:
model.save("models/"+modelname+".h5")
print("model saved!")

model saved!


## （2） epoch 2，学习率1e-5，使用RMSprop

In [11]:
model.compile(optimizer=RMSprop(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1)

Epoch 1/1
491/491 [==============================] - 984s - loss: 0.0966 - acc: 0.9720   


In [12]:
model.save("models/"+modelname+"-2.h5")
print("model saved!")

model saved!


## （3） epoch 3，学习率1e-6，使用RMSprop

In [13]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1)

Epoch 1/1
491/491 [==============================] - 983s - loss: 0.0756 - acc: 0.9785   


In [14]:
model.save("models/"+modelname+"-3.h5")
print("model saved!")

model saved!


## （4） epoch 4，学习率1e-6，使用RMSprop

In [15]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1)

Epoch 1/1
491/491 [==============================] - 983s - loss: 0.0752 - acc: 0.9798   


In [16]:
model.save("models/"+modelname+"-4.h5")
print("model saved!")

model saved!


# 5 输出测试

In [17]:
def predict_test(model,  model_image_size, csv_name):

    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        l.append( [name, *y_pred[i]] )

    l = np.array(l)
    data = {'img': l[:,0]}
    for i in range(10):
        data["c%d"%i] = l[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df = df.sort_values(by='img')
    df.to_csv(csv_name, index=None, float_format='%.3f')
    print("submission saved")

In [18]:
predict_test(model,  model_image_size, "submission/"+modelname+".csv")

4983/4983 [==============================] - 2909s  
submission saved
